In [1]:
import os
import warnings
from langchain._api import LangChainDeprecationWarning
from dotenv import load_dotenv, find_dotenv
from langchain_groq import ChatGroq
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)
_ = load_dotenv(find_dotenv())
llm = ChatGroq(model="llama3-70b-8192")

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/Be_Good.pdf")
doc = loader.load()

In [ ]:
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text = spliter.split_documents(doc)
db = Chroma.from_documents(text, embeddings)
retriever = db.as_retriever()

c:\Users\lenovo\anaconda3\envs\genAI\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answering = create_stuff_documents_chain(llm=llm, prompt=prompt)

rag_chain = create_retrieval_chain(retriever, question_answering)

response = rag_chain.invoke({"input": "What is this article about?"})
response["answer"]

'This article, "Be Good" by Paul Graham, explores the idea that making something people want is more important than focusing on making money, and how this approach can lead to surprising and beneficial outcomes, even in unexpected areas like charity and benevolence.'